# Домашнее задание к лекции «Классификация: Логистическая регрессия и SVM»

In [96]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [97]:
df = pd.read_csv( 'adult.csv', na_values='?')
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


<b>Делаю небольшую предобработку данных:</b>

- Для столбцов, в которых нет пропусков, просто просматриваю уникальные значения на корректность
- Пропуски есть в столбцах workclass, occupation, native-country. В workclass и native-country есть сильно преобладающее значение, и можно заменить пропуски на моду, но это чуть ухудшает качество моделей на выходе. Поэтому просто удаляю строки с пропусками.

In [99]:
# Вспомогательные функции:
    
def moda(column):
    print('Мода: ', column.mode()[0])
    print(column.value_counts())
    
    
def unique(column):
    print('Уникальные значения: ', list(column.unique()))

In [100]:
df_obj = df.select_dtypes(include=[np.object])
df_info = df[df_obj.columns.difference(['fnlwgt', 'capital-gain'])]

for column in df_info:
    print()
    print(column)
    unique(df[column])


education
Уникальные значения:  ['11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th', 'Prof-school', '7th-8th', 'Bachelors', 'Masters', 'Doctorate', '5th-6th', 'Assoc-voc', '9th', '12th', '1st-4th', 'Preschool']

gender
Уникальные значения:  ['Male', 'Female']

income
Уникальные значения:  ['<=50K', '>50K']

marital-status
Уникальные значения:  ['Never-married', 'Married-civ-spouse', 'Widowed', 'Divorced', 'Separated', 'Married-spouse-absent', 'Married-AF-spouse']

native-country
Уникальные значения:  ['United-States', nan, 'Peru', 'Guatemala', 'Mexico', 'Dominican-Republic', 'Ireland', 'Germany', 'Philippines', 'Thailand', 'Haiti', 'El-Salvador', 'Puerto-Rico', 'Vietnam', 'South', 'Columbia', 'Japan', 'India', 'Cambodia', 'Poland', 'Laos', 'England', 'Cuba', 'Taiwan', 'Italy', 'Canada', 'Portugal', 'China', 'Nicaragua', 'Honduras', 'Iran', 'Scotland', 'Jamaica', 'Ecuador', 'Yugoslavia', 'Hungary', 'Hong', 'Greece', 'Trinadad&Tobago', 'Outlying-US(Guam-USVI-etc)', 'France', 'Holan

In [101]:
df = df.dropna() 

df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              45222 non-null  int64 
 1   workclass        45222 non-null  object
 2   fnlwgt           45222 non-null  int64 
 3   education        45222 non-null  object
 4   educational-num  45222 non-null  int64 
 5   marital-status   45222 non-null  object
 6   occupation       45222 non-null  object
 7   relationship     45222 non-null  object
 8   race             45222 non-null  object
 9   gender           45222 non-null  object
 10  capital-gain     45222 non-null  int64 
 11  capital-loss     45222 non-null  int64 
 12  hours-per-week   45222 non-null  int64 
 13  native-country   45222 non-null  object
 14  income           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


<b>Колонки, содержащие строки и имеющие более 2-х независимых значений, перевожу в столбцы 0 и 1 с соответствующими названиями.</b>

<b>education так же использую, как номинальный тип.</b>

In [103]:
X = pd.get_dummies(data=df, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country'])

X.head()

,age,fnlwgt,educational-num,gender,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,Male,0,0,40,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,Male,0,0,50,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,Male,0,0,40,>50K,0,1,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,Male,7688,0,40,>50K,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,198693,6,Male,0,0,30,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,0


<b>Гендер перевожу в 0 и 1</b>

In [104]:
le = LabelEncoder()
X.gender = le.fit_transform(X.gender)

X.head()

,age,fnlwgt,educational-num,gender,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,1,0,0,40,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,1,0,0,50,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,1,0,0,40,>50K,0,1,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,1,7688,0,40,>50K,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,198693,6,1,0,0,30,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,0


<b>Выделяю целевой столбец с доходом и удаляю его из X</b>

In [105]:
Y = pd.Series( data = le.fit_transform(X.income))
Y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [106]:
del X['income']
X.head()

,age,fnlwgt,educational-num,gender,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,1,0,0,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,1,0,0,50,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,1,0,0,40,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,1,7688,0,40,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,34,198693,6,1,0,0,30,0,0,1,...,0,0,0,0,0,0,0,1,0,0


<b>Делю данные на тренировочную и тестовую выборку</b>

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

<b>Строю модель логистической регрессии, вывожу качесво модели на тестовых данных</b>

In [110]:
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.7914381744206616

<b>Строю модель SVM, вывожу качесво модели на тестовых данных</b>

<i>При практически одинаковом качестве, SVM модель обучается намного дольше.</i>

In [111]:
svc = SVC()
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.792145763311516